In [2]:
%%sh
#/opt/conda/bin/python -m pip install --upgrade pip
pip install -q sagemaker --upgrade

  Attempting uninstall: pip
    Found existing installation: pip 21.1.1
    Uninstalling pip-21.1.1:
      Successfully uninstalled pip-21.1.1


/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [5]:
%%sh
aws sts get-caller-identity

{
    "UserId": "AROARPMPLS6TZO7K3PNIN:SageMaker",
    "Account": "101769254823",
    "Arn": "arn:aws:sts::101769254823:assumed-role/AmazonSageMaker-ExecutionRole-20210417T123834/SageMaker"
}


In [20]:
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
role = 'arn:aws:iam::101769254823:role/SageMakerPlatformExecutionRole' #sagemaker.get_execution_role()

2.45.0


In [11]:
%%sh
aws s3 cp s3://lb.athena.queries.result/products_reviews_queries/2021/06/15/07d38fc4-1a3e-401f-9832-7b7529389717.csv /tmp

download: s3://lb.athena.queries.result/products_reviews_queries/2021/06/15/07d38fc4-1a3e-401f-9832-7b7529389717.csv to ../../../../tmp/07d38fc4-1a3e-401f-9832-7b7529389717.csv


In [16]:
filename = '07d38fc4-1a3e-401f-9832-7b7529389717.csv'

In [21]:
prefix = 'amazon-reviews-digital-video-games'

input_data = session.upload_data(path=f'/tmp/{filename}', key_prefix=prefix)

In [22]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.23-1',
                                    role=role,
                                    instance_type= 'ml.m5.2xlarge',
                                    instance_count=1)

In [24]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code= 'preprocessing.py',
    
    inputs= [
        ProcessingInput(
            source=input_data,
            destination='/opt/ml/processing/input'
        )
    ],
    
    outputs=[
        ProcessingOutput(
            output_name='bt_data',
            source='/opt/ml/processing/output/bt'
        ),
        ProcessingOutput(
            output_name='fs_data',
            source='/opt/ml/processing/output/fs'
        )        
    ],
    
    arguments=[
        '--filename', filename        
    ]
    
)


Job Name:  sagemaker-scikit-learn-2021-06-15-08-14-44-663
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-101769254823/amazon-reviews-digital-video-games/07d38fc4-1a3e-401f-9832-7b7529389717.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-101769254823/sagemaker-scikit-learn-2021-06-15-08-14-44-663/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'bt_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-1-101769254823/sagemaker-scikit-learn-2021-06-15-08-14-44-663/output/bt_data', 'LocalPath': '/opt/ml/processing/output/bt', 'S

UnexpectedStatusException: Error for Processing job sagemaker-scikit-learn-2021-06-15-08-14-44-663: Failed. Reason: AlgorithmError: See job logs for more information